### Import libraries

In [1]:
try:
  import yfinance as yf
except:
  !pip install yfinance
  import yfinance as yf
from datetime import date
import datetime
import pandas as pd

**Downloding the data**

In [2]:
# list of all stock
ev_renewable = ["SOLO",
"BLNK",
"PLUG",
"BLDP",
"FCEL"
]

This module gives back the performance over the past days_back days

## Downloading the data 

In [3]:
# downloading data from Yahoo! Finance)
data = yf.download(ev_renewable,'2020-01-01', date.today())

[*********************100%***********************]  5 of 5 completed


In [4]:
def date_difference(date_1, date_2):
    date_to_date = lambda date: ([date.split('-')[0], date.split('-')[1], date.split('-')[2]])
    
    day_1 = date_to_date(date_1)
    day_2 = date_to_date(date_2)

    date_diff = (date(int(day_2[0]), int(day_2[1]), int(day_2[2])) - date(int(day_1[0]), int(day_1[1]), int(day_1[2]))).days
    return date_diff

def trend_finder(date_1, date_2):
    trend = ((data.loc[date_2]["Close"] - data.loc[date_1]["Close"]) / data.loc[date_1]["Close"]) * (100/date_difference(date_1, date_2))
    return trend

def day_by_day_record(df, count_back, interval, date = date.today()):

    """
        Load database and tables
        Args: 
            df (Pandas dataframe): data received from the Yahoo! API
            count_back (Int): number of times to go back
            interval (int): interval between days
            date (DateTime): starting date of 
        Return:
            X (pandas dataframe): A pandas dataframe showing the list of all stocks and their growth
    """
    
    # getting all active stock dates from the downloaded data
    stock_active_dates = [str(date).split('T')[0] for date in list(df.index.values)]

    T = 1
    
    # finding the first valid date
    while True:
        start_date = (date-datetime.timedelta(T)).strftime("%Y-%m-%d")
        
        if start_date in stock_active_dates: 
            start_date = date-datetime.timedelta(T)
            break
        else:
            T += 1
    
    # starting from start date and going back in time until we find enough valid dates
            
    valid_dates = [(date-datetime.timedelta(T)).strftime("%Y-%m-%d")]    
    
    i = 1

    while len(valid_dates) < count_back:
        
        # this day is closer to today
        test_day = (start_date - datetime.timedelta(i*interval+1))

        # checking if the day - interval was an active stock date
        if test_day.strftime("%Y-%m-%d") in stock_active_dates:
            valid_dates.append(test_day.strftime("%Y-%m-%d"))
            
        # fidning the closest day that works 
        else:
            T = 1
            
            while True:
                approx_date = (test_day - datetime.timedelta(T)).strftime("%Y-%m-%d")



                if approx_date in stock_active_dates: 
                    valid_dates.append(approx_date)
                    break

                else:
                    T += 1
                    
        i += 1
        
    trends_dict = {}
    for i in range(len(valid_dates)):
        if i < len(valid_dates)-1:
            date_1 = valid_dates[i]
            date_2 = valid_dates[i+1]
        
        try:
            trend = ((df.loc[date_2]["Close"] - df.loc[date_1]["Close"]) / df.loc[date_1]["Close"]) * (100/date_difference(date_1, date_2))

            day_name = str(date_1) + '   ' + str(date_2)
            trends_dict[day_name] = trend
        except:
            continue

    trends_df = pd.DataFrame(trends_dict) 

    trends_df['average'] = trends_df.mean(axis=1)
    trends_df['standard deviation'] = trends_df.std(axis=1)
    # trends_df['normal average'] = trends_df['average'] / (trends_df['standard deviation']**2)  

    return (trends_df)

In [7]:
# day_by_day_record(df, count_back, interval, start date*default at today*)


# THE CODE BELOW STARTS TODAY AND GOES BACK FOR 30 WEEKS (7 DAYS INTERVALS)
# trends_dataframe_90 = day_by_day_record(data, 30, 7)

# THE CODE BELOW STARTS TODAY AND GOES BACK FOR 12 MONTHS (30 DAYS INTERVALS)
trends_dataframe_12_months = day_by_day_record(data, 12, 30)

trends_dataframe_12_months 

,2021-01-15 2021-01-13,2021-01-13 2021-01-12,2021-01-12 2021-01-11,2021-01-11 2021-01-08,average,standard deviation
BLDP,-9.325336,6.068893,15.715953,0.138120,3.149408,9.098469
BLNK,-3.938432,-2.952175,0.592624,5.185898,-0.278021,3.576386
FCEL,-10.416664,0.470220,17.322831,0.910054,2.071610,9.905433
PLUG,-7.781843,5.007199,18.252039,0.117351,3.898686,9.460190
SOLO,-3.973506,-8.834360,18.489288,2.259107,1.985132,10.308116
